In [1]:
import os, re

%pip install unsloth
%pip install transformers==4.55.4
%pip install timm
import torch; torch._dynamo.config.recompile_limit = 64;

  Using cached unsloth-2025.12.9-py3-none-any.whl.metadata (65 kB)
  Using cached unsloth_zoo-2025.12.7-py3-none-any.whl.metadata (32 kB)
  Using cached xformers-0.0.33.post2-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached diffusers-0.36.0-py3-none-any.whl.metadata (20 kB)
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached torch-2.9.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinu

In [1]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Gemma3N patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.422 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from transformers import TextStreamer
# Helper function for inference
def do_gemma_3n_inference(messages, max_new_tokens = 64):
    # 1. Prepare inputs
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")

    # 2. Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 0.1,
        do_sample = True,
        # Keep the streamer so you can still watch it work in the notebook
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

    # 3. Decode only the NEW tokens (the answer)
    input_length = inputs.input_ids.shape[1]
    new_tokens = outputs[0][input_length:]
    response_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    return response_text

In [7]:
import os
import cv2
import csv
from PIL import Image
from pathlib import Path

def extract_frames_from_path(video_path, num_frames=8):
    """
    Extract evenly spaced frames from a video file.

    Args:
        video_path: Path to video file (string or Path object)
        num_frames: Number of frames to extract (default: 8)

    Returns:
        List of PIL Image objects
    """
    video_path = str(video_path)  # Ensure it's a string
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"Error: Video has 0 frames: {video_path}")
        cap.release()
        return []

    # Calculate the step size (logic from your first source)
    step = max(1, total_frames // num_frames)
    frames = []

    for i in range(num_frames):
        frame_idx = min(i * step, total_frames - 1)  # Ensure we don't exceed total frames
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR (OpenCV) to RGB (PIL) without rotation
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frames.append(img)

    cap.release()

    if len(frames) < num_frames:
        print(f"Warning: Only extracted {len(frames)}/{num_frames} frames from {video_path}")

    return frames


In [8]:
import os
import sys

# Set up paths relative to notebook location
notebook_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in dir() else os.getcwd()
project_root = os.path.dirname(notebook_dir)  # Go up one level from src/
video_folder = os.path.join(project_root, "dataset", "elbow_plank")
output_csv = os.path.join(project_root, "outputs", "gemma3n-4b_inference_results.csv")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

print(f"Project root: {project_root}")
print(f"Looking for videos in: {video_folder}")
print(f"Will save results to: {output_csv}")

results = []

# Check if video folder exists
if not os.path.exists(video_folder):
    print(f"\n⚠️  Warning: Video folder not found: {video_folder}")
    print("Please ensure videos are placed in the dataset/videos/ directory")
    print("Or update the video_folder path above to point to your videos location")
else:
    # Get list of all video files recursively
    video_files = []
    for root, dirs, files in os.walk(video_folder):
        for file in files:
            if Path(file).suffix.lower() in ['.mp4', '.mov', '.avi', '.mkv']:
                video_files.append(Path(root) / file)

    print(f"\nFound {len(video_files)} videos. Starting processing...\n")

    for video_path in video_files:
        relative_path = video_path.relative_to(video_folder)
        print(f"Processing: {relative_path}...")

        # 1. Extract the "Frame Thing" (8 snapshots)
        video_frames = extract_frames_from_path(str(video_path), num_frames=8)

        if not video_frames:
            print(f"  ⚠️  Failed to extract frames from {relative_path}")
            continue

        # 2. Build the message
        messages = [
            {
                "role": "user",
                "content": [
                    *[{"type": "image", "image": img} for img in video_frames],
                    {"type": "text", "text": "Please evaluate the exercise form shown. What mistakes, if any, are present, and what corrections would you recommend?"}
                ],
            },
        ]

        # 3. Inference
        try:
            response = do_gemma_3n_inference(messages, max_new_tokens=256)
            results.append([str(relative_path), response])
            print(f"  ✓ Completed\n")
        except Exception as e:
            print(f"  ✗ Error processing {relative_path}: {e}\n")

    # Save to CSV
    if results:
        with open(output_csv, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["video_name", "model_output"])
            writer.writerows(results)

        print(f"\n✅ Success! Saved {len(results)} results to {output_csv}")
    else:
        print("\n⚠️  No results to save. Please check if videos were processed successfully.")


Project root: /workspace/gemma3-testing
Looking for videos in: /workspace/gemma3-testing/dataset/elbow_plank
Will save results to: /workspace/gemma3-testing/outputs/gemma3n-4b_inference_results.csv

Found 5 videos. Starting processing...

Processing: 00037526.mp4...
## Exercise Form Evaluation: Push-up

The person in the video is performing a push-up. Here's an evaluation of their form, including mistakes and recommended corrections:

**Strengths:**

* **Body Alignment:** The person appears to be maintaining a relatively straight line from head to heels. This is good for engaging the core and preventing injury.
* **Hand Placement:** Hands are placed shoulder-width apart, which is a standard and generally safe position for push-ups.
* **Elbow Angle:** The elbows are bent at a 90-degree angle, which is a good starting point for the push-up.

**Mistakes and Recommended Corrections:**

* **Sagging Hips:** The most noticeable issue is that the hips are sagging towards the floor. This indica